In [11]:
! pip install fuzzywuzzy

In [12]:
import pymysql
import json
import requests
import jellyfish
from datetime import datetime
import re
from difflib import SequenceMatcher as SM
import pandas as pd
from fuzzywuzzy import fuzz

/home/mint/hypenX/InfoGov/.venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
timeout = 10
connection = pymysql.connect(
  charset="utf8mb4",
  connect_timeout=timeout,
  cursorclass=pymysql.cursors.DictCursor,
  db="iia_group2",
  host="world-athletics-world-athletics.h.aivencloud.com",
  password="AVNS_q4qfMZEYhnIhreAFwYi",
  read_timeout=timeout,
  port=24936,
  user="avnadmin",
  write_timeout=timeout,
)


In [27]:
def matcher(col1,col_list):
    m_score = -1
    m_col = None
    for i in col_list:
        sim=SM(isjunk=None, a=col1, b=i).ratio()
        # sim = fuzz.token_set_ratio(col1, i)
        if sim>m_score:
            m_score = sim
            m_col = i
    if(m_score < 0.4):
        return None, m_score
    return m_col, m_score


In [ ]:
cursor = connection.cursor()
cursor.execute("desc GST_Info_PerYear_Data_NS")
x = cursor.fetchall()
sql_fields = [i["Field"] for i in x]
cursor.close()

In [18]:
print(sql_fields)

['id', 'state', 'datee', 'settlement_IGST']


In [25]:
df1 = pd.read_csv("trans_GST_info_per_year_data.csv",index_col=0)

In [26]:
df1

,state,start_date,end_date,Normal tax payers,Composition tax payers,Input service distributor,Casual tax payers,Tax collected at source,Tax deducted at source,Non resident taxpayers,Online information and database access or retrieval services from a place outside india to a non-taxable online recipient,Unique identification number (uin) holders
0,jammu and kashmir,2022-04-01,2023-03-01,110462.0,8499.0,10.0,5.0,151.0,2593.0,0.0,0.0,10.0
1,himachal pradesh,2022-04-01,2023-03-01,99587.0,17679.0,33.0,9.0,207.0,1614.0,0.0,0.0,25.0
2,punjab,2022-04-01,2023-03-01,344609.0,36606.0,83.0,27.0,446.0,3369.0,1.0,0.0,15.0
3,chandigarh,2022-04-01,2023-03-01,27759.0,1609.0,84.0,9.0,214.0,760.0,0.0,0.0,3.0
4,uttarakhand,2022-04-01,2023-03-01,157239.0,34265.0,47.0,6.0,352.0,3480.0,0.0,0.0,11.0
5,haryana,2022-04-01,2023-03-01,482254.0,15470.0,608.0,20.0,941.0,2547.0,4.0,0.0,31.0
6,delhi,2022-04-01,2023-03-01,758918.0,16325.0,1045.0,83.0,1381.0,2842.0,4.0,0.0,316.0
7,rajasthan,2022-04-01,2023-03-01,672438.0,134913.0,115.0,8.0,686.0,19619.0,1.0,0.0,86.0
8,uttar pradesh,2022-04-01,2023-03-01,1377315.0,338879.0,444.0,26.0,1300.0,23645.0,1.0,0.0,234.0
9,bihar,2022-04-01,2023-03-01,484279.0,108500.0,107.0,8.0,374.0,7066.0,0.0,0.0,129.0


In [7]:
schema_incoming_1 = list(df1.keys())
schema_incoming_2 = list(df2.keys())

In [ ]:
schema_map_1 = {}
schema_map_2 = {}

for i in schema_incoming_1:
        schema_map_1[i] = matcher(i,sql_fields)[0]
        if(schema_map_1[i] == None):
                del schema_map_1[i]
           
for i in schema_incoming_2:
        schema_map_2[i] = matcher(i,sql_fields)[0]
        if(schema_map_2[i] == None):
                del schema_map_2[i]


('state', 100)
('datee', 89)
('state', 21)
('settlement_IGST', 17)
('datee', 19)


In [9]:
schema_map_1

{'state': 'state', 'date': 'datee'}

In [41]:
df1

,state,start_date,end_date,Normal tax payers,Composition tax payers,Input service distributor,Casual tax payers,Tax collected at source,Tax deducted at source,Non resident taxpayers,Online information and database access or retrieval services from a place outside india to a non-taxable online recipient,Unique identification number (uin) holders
0,jammu and kashmir,2022-04-01,2023-03-01,110462.0,8499.0,10.0,5.0,151.0,2593.0,0.0,0.0,10.0
1,himachal pradesh,2022-04-01,2023-03-01,99587.0,17679.0,33.0,9.0,207.0,1614.0,0.0,0.0,25.0
2,punjab,2022-04-01,2023-03-01,344609.0,36606.0,83.0,27.0,446.0,3369.0,1.0,0.0,15.0
3,chandigarh,2022-04-01,2023-03-01,27759.0,1609.0,84.0,9.0,214.0,760.0,0.0,0.0,3.0
4,uttarakhand,2022-04-01,2023-03-01,157239.0,34265.0,47.0,6.0,352.0,3480.0,0.0,0.0,11.0
5,haryana,2022-04-01,2023-03-01,482254.0,15470.0,608.0,20.0,941.0,2547.0,4.0,0.0,31.0
6,delhi,2022-04-01,2023-03-01,758918.0,16325.0,1045.0,83.0,1381.0,2842.0,4.0,0.0,316.0
7,rajasthan,2022-04-01,2023-03-01,672438.0,134913.0,115.0,8.0,686.0,19619.0,1.0,0.0,86.0
8,uttar pradesh,2022-04-01,2023-03-01,1377315.0,338879.0,444.0,26.0,1300.0,23645.0,1.0,0.0,234.0
9,bihar,2022-04-01,2023-03-01,484279.0,108500.0,107.0,8.0,374.0,7066.0,0.0,0.0,129.0


In [42]:
for i in df1.keys():
    try:
        df1[schema_map_1[i]] = df1[i]
    except KeyError:
        continue

for i in df2.keys():
    try:
        df2[schema_map_2[i]] = df2[i]
    except KeyError:
        continue

In [43]:
df_1 = df1[list(schema_map_1.values())]
df_2 = df2[list(schema_map_2.values())]

In [44]:
primary_keys = ["state","start_date","end_date"]

In [46]:
schema_map_1

{'state': 'state',
 'start_date': 'start_date',
 'end_date': 'end_date',
 'Normal tax payers': 'UIN_holders',
 'Input service distributor': 'input_service_distributors'}

In [64]:
insert = {}
update = {}
for i in sql_fields[1:]:
    insert[i] = []
    update[i] = []
print(schema_map_1.values())
for  _, i in df_1.iterrows():
    cursor = connection.cursor()
    cursor.execute(
        "select %s from GST_Info_PerYear_Data_NS where state = %s AND start_date = %s AND end_date = %s",
        (
            ','.join(list(schema_map_1.values())),
            i["state"],
            i["start_date"],
            i["end_date"]
            )
    )
    x = cursor.fetchall()
    if(len(x) == 0):
        for j in insert.keys():
            try:
                insert[j].append(i[j])
            except KeyError:
                insert[j].append(None)
    else:
        x = x[0]
        for j in insert.keys():
            try:
                if(x[j] == None):
                    update[j].append(i[j])
                elif(x[j] != i[j]):
                    update[j].append(i[j])
                else:
                    update[j].append(None)
            except KeyError:
                update[j].append(None)
    cursor.close()

dict_values(['state', 'start_date', 'end_date', 'UIN_holders', 'input_service_distributors'])


In [65]:
pd.DataFrame(insert)

,state,start_date,end_date,input_service_distributors,UIN_holders,settlement_IGST
0,jammu and kashmir,2022-04-01,2023-03-01,10.0,110462.0,None
1,himachal pradesh,2022-04-01,2023-03-01,33.0,99587.0,None
2,punjab,2022-04-01,2023-03-01,83.0,344609.0,None
3,chandigarh,2022-04-01,2023-03-01,84.0,27759.0,None
4,uttarakhand,2022-04-01,2023-03-01,47.0,157239.0,None
5,haryana,2022-04-01,2023-03-01,608.0,482254.0,None
6,delhi,2022-04-01,2023-03-01,1045.0,758918.0,None
7,rajasthan,2022-04-01,2023-03-01,115.0,672438.0,None
8,uttar pradesh,2022-04-01,2023-03-01,444.0,1377315.0,None
9,bihar,2022-04-01,2023-03-01,107.0,484279.0,None


In [66]:
for _, i in pd.DataFrame(insert).iterrows():
    try:
        cursor = connection.cursor()
        cursor.execute(
            "insert into GST_Info_PerYear_Data_NS ("+', '.join(sql_fields[1:])+") values ("+ ', '.join(["%s" for i in sql_fields[1:]]) +")",
            tuple(i.to_list())
        )
        connection.commit()
        cursor.close()
    except:
        continue

for _, i in pd.DataFrame(update).iterrows():
    try:
        cursor = connection.cursor()
        final  = []
        final_tup = []
        for j in i.keys():
            if(i[j] != None):
                final.append(' '.join([str(j),"=","%s"]))
                final_tup.append(i[j])
        final = ', '.join(final)
        final_tup.extend([i["state"],i["start_date"],i["end_date"]])
        # print(final_tup)
        cursor.execute(
            f"update GST_Info_PerYear_Data_NS set {final} where state = %s AND start_date = %s AND end_date = %s",
            tuple(final_tup)
        )
        connection.commit()
        cursor.close()
    except:
        continue


In [67]:
df2

,state,start_date,end_date,Settlement of Integrated goods and services tax ( IGST )
0,andaman and nicobar islands,2017-01-01,2018-12-01,7.008180
1,andaman and nicobar islands,2018-01-01,2019-12-01,12.980731
2,andaman and nicobar islands,2019-01-01,2020-12-01,17.973762
3,andaman and nicobar islands,2020-01-01,2021-12-01,16.757644
4,andaman and nicobar islands,2021-01-01,2022-12-01,18.292241
...,...,...,...,...
211,west bengal,2018-01-01,2019-12-01,754.974497
212,west bengal,2019-01-01,2020-12-01,832.769101
213,west bengal,2020-01-01,2021-12-01,750.964769
214,west bengal,2021-01-01,2022-12-01,967.482461


In [71]:
insert = {}
update = {}
for i in sql_fields[1:]:
    insert[i] = []
    update[i] = []
print(schema_map_2.values())
for  _, i in df_2.iterrows():
    cursor = connection.cursor()
    cursor.execute(
        "select * from GST_Info_PerYear_Data_NS where state = %s AND start_date = %s AND end_date = %s",
        (
            i["state"],
            i["start_date"],
            i["end_date"]
        )
    )
    x = cursor.fetchall()
    # print(i,x)
    if(len(x) == 0):
        for j in insert.keys():
            try:
                insert[j].append(i[j])
            except KeyError:
                insert[j].append(None)
    else:
        x = x[0]
        for j in update.keys():
            try:
                if(x[j] == None and j not in primary_keys):
                    update[j].append(i[j])
                elif(x[j] != i[j] or j in primary_keys):
                    update[j].append(i[j])
                else:
                    update[j].append(None)
            except KeyError:
                update[j].append(None)
    cursor.close()

dict_values(['state', 'start_date', 'end_date'])


In [72]:
pd.DataFrame(update)

,state,start_date,end_date,input_service_distributors,UIN_holders,settlement_IGST


In [73]:
for _, i in pd.DataFrame(insert).iterrows():
    try:
        cursor = connection.cursor()
        cursor.execute(
            "insert into GST_Info_PerYear_Data_NS ("+', '.join(sql_fields[1:])+") values ("+ ', '.join(["%s" for i in sql_fields[1:]]) +")",
            tuple(i.to_list())
        )
        connection.commit()
        cursor.close()
    except:
        continue

for _, i in pd.DataFrame(update).iterrows():
    try:
        cursor = connection.cursor()
        final  = []
        final_tup = []
        for j in i.keys():
            if(i[j] != None):
                final.append(' '.join([str(j),"=","%s"]))
                final_tup.append(i[j])
        final = ', '.join(final)
        final_tup.extend([i["state"],i["start_date"],i["end_date"]])
        cursor.execute(
            f"update GST_Info_PerYear_Data_NS set {final} where state = %s AND start_date = %s AND end_date = %s",
            tuple(final_tup)
        )
        connection.commit()
        cursor.close()
    except:
        continue


In [104]:
datetime.strptime(i["start_date"], "%Y-%m-%d").date()

datetime.date(2017, 4, 1)